In [14]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import json
import routeExtract as rE
import monte_carlo_routedelay as m
# API Keys
API_KEY_TRANSIT = "SE8BzcNeqwzk2XIkWJbAcKE0m27BIbTB2fzwSVfEOAE"
API_KEY_TRAFFIC = "MSdwf2FTH56aUQ58-vpWF3sI2FC1dNtNWiTUX4sL8Yg"

def run_simulation(start, end):
    bus_coords = rE.extract_transit_steps(API_KEY_TRANSIT, start, end)

    delays = []
    confidences = []

    for lat, lng in bus_coords:
        response = m.get_traffic_data(API_KEY_TRAFFIC, lat, lng)
        delay, conf = m.process_response(response)
        delays.append(delay)
        confidences.append(conf)

    total_delay = sum(delays)
    average_confidence = sum(confidences) / len(confidences) if confidences else 0

    simulated_totals = []
    for _ in range(m.NUM_ITERATIONS):
        if m.random.random() <= average_confidence:
            simulated_totals.append(total_delay)
        else:
            simulated_totals.append(0)

    min_delay = min(simulated_totals)
    max_delay = max(simulated_totals)
    avg_delay = sum(simulated_totals) / m.NUM_ITERATIONS

    return min_delay, max_delay, avg_delay, average_confidence


API_KEY = "SE8BzcNeqwzk2XIkWJbAcKE0m27BIbTB2fzwSVfEOAE"

# --- Main Window ---
root = tk.Tk()
root.title("Route Finder")
root.geometry("700x500")

# --- Title Label ---
title_label = tk.Label(root, text="Multimodal Route Finder", font=("Arial", 16))
title_label.pack(pady=10)

# --- Start Location Input ---
start_frame = tk.Frame(root)
start_frame.pack(pady=5)
tk.Label(start_frame, text="Start Location (lat, lon):").pack(side="left")
start_entry = tk.Entry(start_frame, width=30)
start_entry.pack(side="left")

# --- End Location Input ---
end_frame = tk.Frame(root)
end_frame.pack(pady=5)
tk.Label(end_frame, text="End Location (lat, lon):").pack(side="left")
end_entry = tk.Entry(end_frame, width=30)
end_entry.pack(side="left")

# --- Display Area (can show output later) ---
display_label = tk.Label(root, text="Route summary will appear here.", wraplength=350, justify="left")
display_label.pack(pady=20)


# --- Function ---
def on_get_route():
    start = start_entry.get()
    end = end_entry.get()

    try:
        coords = rE.extract_transit_steps(API_KEY, start, end)
        buses = rE.extract_bus_numbers(API_KEY, start, end)
        directions = rE.extract_bus_directions(API_KEY, start, end)

        summary = f"Start: {start}\nEnd: {end}\n\n"
        summary += "Bus Stops:\n" + "\n".join([f"{lat}, {lon}" for lat, lon in coords]) + "\n\n"
        summary += "Bus Numbers:\n" + ", ".join(buses) + "\n\n"
        summary += "Directions:\n" + ", ".join([d if d else "unknown" for d in directions])

        display_label.config(text=summary)

    except Exception as e:
        display_label.config(text=f"Error fetching route: {str(e)}")

def on_simulate_delay():
    start = start_entry.get()
    end = end_entry.get()

    try:
        min_d, max_d, avg_d, conf = run_simulation(start, end)

        delay_result = f"\n\n🚌 Monte Carlo Simulation of Bus Delays:\n"
        delay_result += f"🔻 Min Delay: {min_d:.2f} sec\n"
        delay_result += f"🔺 Max Delay: {max_d:.2f} sec\n"
        delay_result += f"📊 Avg Delay: {avg_d:.2f} sec\n"
        delay_result += f"🧠 Avg Confidence: {conf:.2f}"

        current_text = display_label.cget("text")  # get existing UI content
        display_label.config(text=current_text + delay_result)

    except Exception as e:
        current_text = display_label.cget("text")
        display_label.config(text=current_text + f"\n\nSimulation Error: {str(e)}")

get_route_btn = tk.Button(root, text="Get Route", command=on_get_route)
get_route_btn.pack(pady=10)
simulate_btn = tk.Button(root, text="Simulate Traffic Delay", command=on_simulate_delay)
simulate_btn.pack(pady=5)

# --- Run the app ---
root.mainloop()


KeyboardInterrupt: 